# Import

In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraForPreTraining, ElectraTokenizerFast, ElectraConfig
from torch.optim import AdamW

# hyper parameters
epochs = 5
learning_rate = 5e-5
batch_size = 4

# RecAdam optimizer

In [2]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""RecAdam optimizer"""

import logging
import math
import numpy as np

import torch
from torch.optim import Optimizer


logger = logging.getLogger(__name__)


def anneal_function(function, step, k, t0, weight):
    if function == 'sigmoid':
        return float(1 / (1 + np.exp(-k * (step - t0)))) * weight
    elif function == 'linear':
        return min(1, step / t0) * weight
    elif function == 'constant':
        return weight
    else:
        ValueError


class RecAdam(Optimizer):
    """ Implementation of RecAdam optimizer, a variant of Adam optimizer.

    Parameters:
        lr (float): learning rate. Default 1e-3.
        betas (tuple of 2 floats): Adams beta parameters (b1, b2). Default: (0.9, 0.999)
        eps (float): Adams epsilon. Default: 1e-6
        weight_decay (float): Weight decay. Default: 0.0
        correct_bias (bool): can be set to False to avoid correcting bias in Adam (e.g. like in Bert TF repository). Default True.
        anneal_fun (str): a hyperparam for the anneal function, decide the function of the curve. Default 'sigmoid'.
        anneal_k (float): a hyperparam for the anneal function, decide the slop of the curve. Choice: [0.05, 0.1, 0.2, 0.5, 1]
        anneal_t0 (float): a hyperparam for the anneal function, decide the middle point of the curve. Choice: [100, 250, 500, 1000]
        anneal_w (float): a hyperparam for the anneal function, decide the scale of the curve. Default 1.0.
        pretrain_cof (float): the coefficient of the quadratic penalty. Default 5000.0.
        pretrain_params (list of tensors): the corresponding group of params in the pretrained model.
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-6, weight_decay=0.0, correct_bias=True,
                 anneal_fun='sigmoid', anneal_k=0, anneal_t0=0, anneal_w=1.0, pretrain_cof=5000.0, pretrain_params=None):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {} - should be >= 0.0".format(lr))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[1]))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {} - should be >= 0.0".format(eps))
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, correct_bias=correct_bias,
                        anneal_fun=anneal_fun, anneal_k=anneal_k, anneal_t0=anneal_t0, anneal_w=anneal_w,
                        pretrain_cof=pretrain_cof, pretrain_params=pretrain_params)
        super().__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()
        for group in self.param_groups:
            for p, pp in zip(group["params"], group["pretrain_params"]):
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError("Adam does not support sparse gradients, please consider SparseAdam instead")

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state["step"] = 0
                    # Exponential moving average of gradient values
                    state["exp_avg"] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state["exp_avg_sq"] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state["exp_avg"], state["exp_avg_sq"]
                beta1, beta2 = group["betas"]

                state["step"] += 1

                # Decay the first and second moment running average coefficient
                # In-place operations to update the averages at the same time
                exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1.0 - beta2, grad, grad)
                denom = exp_avg_sq.sqrt().add_(group["eps"])

                step_size = group["lr"]
                if group["correct_bias"]:
                    bias_correction1 = 1.0 - beta1 ** state["step"]
                    bias_correction2 = 1.0 - beta2 ** state["step"]
                    step_size = step_size * math.sqrt(bias_correction2) / bias_correction1

                # With RecAdam method, the optimization objective is
                # Loss = lambda(t)*Loss_T + (1-lambda(t))*Loss_S
                # Loss = lambda(t)*Loss_T + (1-lambda(t))*\gamma/2*\sum((\theta_i-\theta_i^*)^2)
                if group['anneal_w'] > 0.0:
                    # We calculate the lambda as the annealing function
                    anneal_lambda = anneal_function(group['anneal_fun'], state["step"], group['anneal_k'],
                                                    group['anneal_t0'], group['anneal_w'])
                    assert anneal_lambda <= group['anneal_w']
                    # The loss of the target task is multiplied by lambda(t)
                    p.data.addcdiv_(-step_size * anneal_lambda, exp_avg, denom)
                    # Add the quadratic penalty to simulate the pretraining tasks
                    p.data.add_(-group["lr"] * (group['anneal_w'] - anneal_lambda) * group["pretrain_cof"], p.data - pp.data)
                else:
                    p.data.addcdiv_(-step_size, exp_avg, denom)

                # Just adding the square of the weights to the loss function is *not*
                # the correct way of using L2 regularization/weight decay with Adam,
                # since that will interact with the m and v parameters in strange ways.
                #
                # Instead we want to decay the weights in a manner that doesn't interact
                # with the m/v parameters. This is equivalent to adding the square
                # of the weights to the loss with plain (non-momentum) SGD.
                # Add weight decay at the end (fixed version)
                if group["weight_decay"] > 0.0:
                    p.data.add_(-group["lr"] * group["weight_decay"], p.data)

        return loss

## Create test Dataset and Dataloader

In [3]:
tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")

class JsonlDataset(Dataset):
    def __init__(self, filename, is_test=False):
        self.data = [json.loads(line) for line in open(filename, 'r', encoding='utf-8')]
        self.is_test = is_test

    def __len__(self):
        return len(self.data)
    
    def getMerged(self, question, options, article):
        return [
            f"{question.replace('@placeholder', options[i])} [SEP] {article}" for i in range(5)
        ]
    
    def __getitem__(self, idx):
        item = self.data[idx]
        article = item['article']
        question = item['question']
        options = [item[f'option_{i}'] for i in range(5)]
        
        merged = self.getMerged(question, options, article)
        
        _input = tokenizer(
            merged,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        
        input_ids = _input["input_ids"]
        attention_mask = _input["attention_mask"]
        
        origin = f"{question.replace('@placeholder', '[MASK]')} [SEP] {article}"
        
        origin_ids = tokenizer(
            origin,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )["input_ids"]
        
        option_position = (origin_ids == tokenizer.mask_token_id).nonzero().tolist()[0][1]
        
        labels = torch.zeros(5, 512)
        for i in range(5):
            if i != item['label']:
                labels[i][option_position] = 1
        
        if self.is_test:
            return {
                'article': article,
                'question': question,
                'options': options,
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'merged': merged
            }
        else:
            return {
                'article': article,
                'question': question,
                'options': options,
                'label': item['label'],
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'labels': labels,
                'merged': merged
            }

In [4]:
task1_data_path = {
    'train_data_path': '../input/semevaldataset/training_data/Task_1_train.jsonl',
    'dev_data_path': '../input/semevaldataset/training_data/Task_1_dev.jsonl',
    'test_data_path': '../input/semevaldataset/trail_data/Task_1_Imperceptibility.jsonl'
}

task2_data_path = {
    'train_data_path': '../input/semevaldataset/training_data/Task_2_train.jsonl',
    'dev_data_path': '../input/semevaldataset/training_data/Task_2_dev.jsonl',
    'test_data_path': '../input/semevaldataset/trail_data/Task_2_Nonspecificity.jsonl'
}

# Task 1
task1_train_dataset = JsonlDataset(task1_data_path['train_data_path'])
task1_train_loader = DataLoader(task1_train_dataset, batch_size=batch_size, shuffle=True)

task1_dev_dataset = JsonlDataset(task1_data_path['dev_data_path'])
task1_dev_loader = DataLoader(task1_dev_dataset, batch_size=batch_size, shuffle=True)

# task1_test_dataset = JsonlDataset(task1_data_path['test_data_path'])
# task1_test_loader = DataLoader(task1_test_dataset, batch_size=batch_size, shuffle=True)

# Task 2
task2_train_dataset = JsonlDataset(task2_data_path['train_data_path'])
task2_train_loader = DataLoader(task2_train_dataset, batch_size=batch_size, shuffle=True)

task2_dev_dataset = JsonlDataset(task2_data_path['dev_data_path'])
task2_dev_loader = DataLoader(task2_dev_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def train(model, train_loader, dev_loader, output_path):
    best_val_loss = float('inf')
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    optimizer = RecAdam(model.parameters(), pretrain_params=model.parameters(), lr=learning_rate)
    model = model.to(device)
    
    for epoch in range(epochs):
        total_loss = 0
        
        model.train()
        for step, batch in enumerate(train_loader):
            _input_ids = batch['input_ids'].to(device)
            _attention_mask = batch['attention_mask'].to(device)
            _labels = batch['labels'].to(device)
            _label = batch['label']

            _input_ids = _input_ids.view(-1, _input_ids.shape[2]) # 20, 512
            _labels = _labels.view(-1, _labels.shape[2])
            _attention_mask = _attention_mask.view(-1, _attention_mask.shape[2])

            if step % 100 == 0 and not step == 0:
                print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_loader)))
                
            for i in range(_input_ids.shape[0] // 5):
                optimizer.zero_grad()
                
                input_ids = _input_ids[i * 5 : (i + 1) * 5]
                labels = _labels[i * 5 : (i + 1) * 5].float()
                attention_mask = _attention_mask[i * 5 : (i + 1) * 5]

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

                loss = outputs.loss
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader) / 5
        print(f'Training loss: {avg_train_loss}')
                        
        model.eval()                
        total_val_loss = 0
        for batch in dev_loader:
            with torch.no_grad():
                _input_ids = batch['input_ids'].to(device)
                _attention_mask = batch['attention_mask'].to(device)
                _labels = batch['labels'].to(device)
                _label = batch['label']
                
                _input_ids = _input_ids.view(-1, _input_ids.shape[2]) # 20, 512
                _labels = _labels.view(-1, _labels.shape[2])
                _attention_mask = _attention_mask.view(-1, _attention_mask.shape[2])

                for i in range(_input_ids.shape[0] // 5):
                    input_ids = _input_ids[i * 5 : (i + 1) * 5]
                    labels = _labels[i * 5 : (i + 1) * 5].float()
                    attention_mask = _attention_mask[i * 5 : (i + 1) * 5]

                    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

                    loss = outputs.loss

                    total_val_loss += loss.item()
                    
        avg_val_loss = total_val_loss / len(dev_loader) / 5
        print(f'Validation loss: {avg_val_loss}')

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            model.save_pretrained(output_path)
            print(f'{output_path} saved.')
        
        model.save_pretrained(output_path.replace('best', 'last'))

In [ ]:
discriminator = ElectraForPreTraining.from_pretrained("google/electra-large-discriminator")

train(discriminator, task1_train_loader, task1_dev_loader, '../working/task1_electra_recadam_best.bin')
train(discriminator, task2_train_loader, task2_dev_loader, '../working/task2_electra_recadam_best.bin')